# Chapter 18: Microstructure Proxy Features
AFML focus: liquidity/impact proxies and informational frictions.


In [1]:
import math
import matplotlib.pyplot as plt
import polars as pl
import openquant
import sys
from pathlib import Path
sys.path.insert(0, str(Path('notebooks/python/scripts').resolve()))
from afml_chapter_utils import (
    fetch_panel,
    simple_returns,
    probs_and_sides_from_momentum,
    timestamps_from_dates,
    lag_corr,
    fracdiff_ffd,
)

panel = fetch_panel(window=900)
dates = panel['date'].to_list()
uso = panel['USO'].to_list()
uso_ret = simple_returns(uso)
probs, sides = probs_and_sides_from_momentum(uso)
timestamps = timestamps_from_dates(dates)
asset_names = ['USO', 'BNO', 'XLE', 'GLD', 'UNG']
asset_prices = panel.select(asset_names).rows()
print('rows', panel.height, 'range', dates[0], dates[-1])


rows 900 range 2022-07-08 2026-02-06


In [2]:
vol = panel['USO_volume'].to_list()
ret = uso_ret
amihud_proxy = [0.0 if v == 0 else abs(r)/v for r,v in zip(ret, vol)]

sig_df = openquant.adapters.to_polars_signal_frame(timestamps, probs, side=sides, symbol='USO')

plt.figure(figsize=(10,3))
plt.plot(amihud_proxy[-300:], lw=1)
plt.title('Chapter 18: Amihud-style Illiquidity Proxy (USO)')
plt.tight_layout()
plt.show()

print('signal frame rows', sig_df.height, 'proxy mean', sum(amihud_proxy)/len(amihud_proxy))


signal frame rows 900 proxy mean 3.9197357212120375e-09


## Interpretation
Even with proxy-level microstructure features, we can monitor changing liquidity burden and feed this into cost-aware gating.

